# ___Les types basiques___

___

- Dans sa forme la plus simple, déclarer une variable se fait avec le mot-clef `let` suivi de son `nom` puis de deux petits points `:`, de son `type` et enfin `= sa valeur`. **En Rust, la syntaxe impose que la déclaration du type suit le nom de la variable et non le contraire**, ceci peut être, au départ, extremement déroutant.

 Quelque particularité à connaître du langage Rust sur ses types:
 > - Pas de cast implicite.
 > - Il est impossible d'utiliser une variable non-initialisée, sauf en cas de code unsafe.
 > - Grace a son **moteur d'inférence**, Rust déduit les types qui n'ont pas été explicitement déclarés.

## Les types numériques

- Sans grande surprise, ces types se manipulent à l'aide des opérateurs numériques Addition, Soustraction, Multiplication, Division et Modulo.

| Non-Signé	 | Signé  |  Flottants | Représentation |
-------- |----------|----------|---------|
| u8	 |   i8	    |          |   8 bits |
| u16	 |   i16	|          |  16 bits |
| u32	 |   i32	|   f32    |  32 bits |
| u64	 |   i64	|   f64    |  64 bits |
| u128	 |   i128	|          | 128 bits |
| usize	 |   isize	|          | register size |

**NB** *Les types f80 et f128 n'existent pas en Rust pour des raisons de portage de code entre les différentes architectures.*

### Déclaration

In [2]:
let a: u32 = 42;
let b: i64 = -127;
let c: isize = -128;
let pi: f32 = std::f32::consts::PI;
let pi: f64 = std::f64::consts::PI;

- Il n'est pas toujours utile de définir explicitement les types en Rust, le moteur d'inférence de type devine quel est le type le plus évident. Par exemple, si j'ajoute un type inféré  à un type connu par le compilateur tel u64, le moteur d'inférence de Rust déduira que mon premier type est un u64.

In [3]:
let t1: u64 = 42;
let t2 = 12; // Ici le type n'est pas explicitement donné
let t3 = t1 + t2; // le type de t3 sera un u64

- C'est aussi courant lorsque le type est retourné par une fonction (L'annotation du type étant obligatoire lors de l'écriture des prototypes de fonctions).

In [4]:
fn return_u64() -> u64 { // Cette fonction retourne un u64
    12
}
let t4 = return_u64(); // Ici t4 sera un u64

- Quand il y a une erreur dans le code, le moteur d'inférence ne pourra pas déduire le bon type !

In [5]:
fn return_u32() -> u32 {
    12
}
let t5 = return_u64() + return_u32();

Error: mismatched types

Error: cannot add `u32` to `u64`

- Sanns annotation de type, Rust utilise un type par défaut.

In [6]:
let unknown_unsigned_type = 12; // i32
let unknown_signed_type = -12; // i32
let unknown_float = 3.12; // f64

- Il peut arriver que l'annotation de type soit obligatoire

In [7]:
fn main() {
    let s = "11";
    dbg!(s.parse()); // la macro dbg! permet de "debugger" la valeur
}

Error: type annotations needed

**Le compilateur ne sait pas vers quel type convertir la chaine de caractere "11"**

In [8]:
fn main() {
    let s = "11";
    dbg!(s.parse::<u32>()); // On precise que l'on veut un u32
}
main()

[src/lib.rs:7] s.parse::<u32>() = Ok(
)
    11,


()

**Comment se nomme la syntaxe `::<u32>` ?**

On appelle cela un **turbofish**, c'est une façon plutôt classe d'annoter les types bien que ce ne soit pas généralement indispensable. Nous aurions aussi pu écrire:

In [9]:
fn main() {
    let s = "11";
    let number: Result<u32, _> = s.parse(); // On precise que l'on veut un u32
    dbg!(number);
}
main()

[src/lib.rs:8] number = Ok(
    11,
)


()

### Différentes façons de donner les valeurs


| Number literals | Example |
-------|-------|
| Decimal |	98_222 |
| Hex	| 0xff |
| Octal	| 0o77 |
| Binary |	0b1111_0000 |
| Byte (u8 only)	| b'A' |

*L'underscore `_` peut être utilise pour la lisibilité des valeurs*.

### La casting ou conversion entre les types

- Pour passer d'un type primitif a l'autre, il est possible d'utiliser le mot clef `as`

In [10]:
let mon_petit_entier: u32 = 42;
let mon_grand_entier: u64 = mon_petit_entier as u64;

let mon_petit_flottant: f32 = 1.12;
let mon_grand_flottant: f64 = mon_petit_flottant as f64;

- Les conversions en RUST doivent être toujours explicites ! Le mot-clef `as` doit donc être toujours utilisé. C'est une sécurité du langage.

In [11]:
let mon_petit_entier: u32 = 42;
let mon_grand_entier: u64 = mon_petit_entier;

Error: mismatched types

- Ainsi l'utilisateur assume ce qu'il fait lorsqu'il utilise `as`...

In [12]:
    let v: u64 = 7_212_342_332;
    let u = v as u32;
    dbg!(u);

[src/lib.rs:144] u = 2917375036


## Les booléens

- Ce type se manipule avec les opérateurs logiques habituels AND, OR, XOR et NOT.

In [13]:
let b: bool = false;
let c: bool = true;

let mut b: bool = false;
if !b {
    println!("B is false");
}
b = true;
if b {
     println!("B is true");
}
let c: bool = true;
if b & c {
    println!("B and C are true")
}

B is false
B is true
B and C are true


()

- Notez qu'à cause de problèmes bien connus dans d'autres langages, Rust ne permet pas le cast implicite entres les types numériques et les booléens. Seul true ou false peuvent être utilisés.

In [14]:
let b: bool = 0;

Error: mismatched types

In [15]:
let v: u32 = 42;
let d: bool = v as bool;

Error: cannot cast as `bool`

In [16]:
let v: u32 = 42;
let d: bool = v != 0;
dbg!(d);

[src/lib.rs:146] d = true


## Les caractères

- Le type caractère est au format utf8 en Rust, ceci doit etre pris en consideration car il peut y avoir des bugs lors du portage de ce type vers d'autres langages (Ce n'est pas des caractères ASCII simples comme le type `char` en C).

In [17]:
let c: char = 't';
println!("len: {} - {}", c.len_utf8(), c);
let c: char = 'ф';
println!("len: {} - {}", c.len_utf8(), c);
let c: char = '錆';
println!("len: {} - {}", c.len_utf8(), c);

len: 1 - t
len: 2 - ф
len: 3 - 錆


## Les chaines de caractères &str

- Les chaînes de caractères en Rust ressemblent a celles d'autres langages, elles s'écrivent entre guillemets doubles. Notez que ce sont des chaînes de caractère utf8 toujours valides et elles sont aussi constantes, car elles sont présentes dans le code du programme. Elles sont du type `&str`.

In [18]:
let ma_chaine = "Banane";
let ma_seconde_chaine: &str = "Banane";

- Notez que si on veut les écrire sur plusieurs lignes, il est possible d'utiliser le caractère d'échappement `\`.

In [19]:
let ma_chaine_echapee = "Banane\
Volante";
dbg!(ma_chaine_echapee);

[src/lib.rs:152] ma_chaine_echapee = "BananeVolante"


> Nous verrons plus tard comment allouer dynamiquement de la mémoire pour des chaînes de caractère non-constantes.

## Les Types composés

- On dit qu'une variable est de type composé lorsque qu'elle contient un ensemble de valeurs de plusieurs types. Il existe le cas des **tuple** ou couple et celui des **array** ou tableaux.

### Le cas des array

- Les tableaux sont une collection de **n variables de même type**, ils sont définis avec une taille précise. On utilise la notation **crochets** `[    ]` pour déclarer un type tableau et chaque valeur associée sera séparée par une virgule `[0, 1, 2, ...]`. Un tableau peut aussi etre initialisé avec cette syntaxe `[value; nbr_elements]`. Enfin, on utilisera aussi la notation crochet pour acceder a chacun des champs `mon_array[3]`.

In [4]:
// Un tableau initialisé avec plusieurs i32.
let xs: [i32; 5] = [1, 2, 3, 4, 5];

// Tous les éléments sont initialisés a 0
let ys: [i32; 500] = [0; 500];

// Accés aux éléments
println!("First element of the array: {}", xs[0]);
println!("Second element of the array: {}", xs[1]);

// Taille implicite
let xz = ['a', 'b', 'c'];

// Tableau de chaines de caractere
let categories: [&str; 4] = ["Terre", "Feu", "Air", "Eau"];
println!("Acces a la seconde case du tableau : {}", categories[1]);

First element of the array: 1
Second element of the array: 2
Acces a la seconde case du tableau : Feu


> **NB** Le programme panique lors d'un acces illicite au tableau.

In [21]:
dbg!(xz[6]);

thread '<unnamed>' panicked at 'index out of bounds: the len is 3 but the index is 6', src/lib.rs:158:6
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
   2: core::panicking::panic_bounds_check
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:159:5
   3: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   4: run_user_code_15
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


> *Le programme panique a l'exécution (ou bien à la compilation si l'on utilise des const fn), en paniquant il s'arrête, et affiche a l'écran la raison du panique ainsi que la backtrace.* **Le panique n'est pas un crash**. *Il n'y a pas de segmentation fault, le programme s'arrête tout simplement, car quelque chose d'impossible, d'illogique ou de dangereux allait être exécuté. Imaginez en C le même bout de code avec cet accès a la case 6 du tableau, quels auraient été les conséquences ?* 

### Le cas des tuples

- Un tuple s'écrit en mettant plusieurs types entre parenthèses séparés par une virgule, un tuple peut aussi être vide ou bien ne contenir qu'un seul type. L'avantage des tuples c'est qu'ils peuvent contenir des variables de type différents. Comme les tableaux, ils sont de taille fixe. On utilise la notation `.` puis numero du champs pour acceder aux elements du tuple comme par exemple `mon_tuple.0` ou `mon_tuple.1`

In [22]:
let tuple1: (u32, u16) = (3, 6); // Initialisation d'un tuple
let tuple2 = (12, 3.14); // Declaration des types implicites

dbg!(tuple2.1);

fn mul(n: (u32, f64), e: u32) -> (u32, f64) {
    (n.0 * e, n.1 * e as f64)
}
let g = mul((42, std::f64::consts::PI), 2);    
dbg!(g);

[src/lib.rs:164] tuple2.1 = 3.14
[src/lib.rs:167] g = (
    84,
    6.283185307179586,
)


- Le programme panique tout autant lors d'un acces illicite.

In [23]:
let tuple3: (f64, f64) = (1.2, 1.7);
dbg!(tuple3.2);

Error: no field `2` on type `(f64, f64)`

## Les alias de Type

- Tout comme en C, il est possible de nommer ses propres alias de type. C'est l'équivalent de Typedef :

In [12]:
type Tuple = (u32, f64); // Ici, Tuple es un alias de du tuple (u32, f64)
fn mul2(n: Tuple, e: u32) -> Tuple { // Cette fonction prend l'alias Tuple en parametre
    (n.0 * e, n.1 * e as f64)
}
{
    let tuple: Tuple = (42, std::f64::consts::PI); // Declaration d'une variable de type Tuple
    let g = mul2(tuple, 2);    
    dbg!(g);
}

[src/lib.rs:134] g = (
    6.283185307179586,
    84,
)


()

- Cependant, les alias de type **ne sont pas des constructeurs.** Ce qui implique que la syntaxe commune aux constructeurs dans d'autres langages `Type(parameters...)` n'est pas valide :

In [9]:
{
    let g = mul2(Tuple(42, std::f64::consts::PI), 2);    
    dbg!(g);
}

Error: expected function, tuple struct or tuple variant, found type alias `Tuple`

> Le kernel rust de Jupyter ne le dit pas, mais quand je tente de compiler le même code en local sur un fichier, le message d'erreur du compilateur est plus éloquent :
```
error[E0423]: expected function, tuple struct or tuple variant, found type alias `Tuple`
  --> src/main.rs:45:18
   |
45 |     let g = mul2(Tuple(42, std::f64::consts::PI), 2);
   |                  ^^^^^
   |
   = note: can't use a type alias as a constructor
```
**note: can't use a type alias as a constructor**

> Apparemment, le compilateur a bien compris ce que j'ai tenté de faire !

## Exercices

> corriger les bouts de code ci-dessous: Appliquer les modifications necessaires puis lancer run.

### Types incompatibles

In [24]:
let v1: u32 = 42;

In [25]:
let v2: i64 = 42; // FIX AND RUN IT

In [26]:
let result: u32 = v1 + v2; // RUN IT
dbg!(result);

Error: mismatched types

Error: cannot add `i64` to `u32`

**SOLUTION**

**Le type de la variable v2 doit être un u32, ne déclarer aucun type est possible aussi, le moteur d'inférence fonctionnera correctement.**

### To be bool or not bool

In [27]:
let mon_boolean: bool = 42; // FIX AND RUN IT

Error: mismatched types

**SOLUTION**

**Un booléen (bool) ne peut être initialisé en Rust qu'a true ou a false, jamais avec un entier.**

### Out of array bounds

In [28]:
let array = [1, 2, 3]; // FIX AND RUN IT

In [29]:
dbg!(array[0] + array[1] + array[2] + array[3]); // RUN IT

thread '<unnamed>' panicked at 'index out of bounds: the len is 3 but the index is 3', src/lib.rs:173:39
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
   2: core::panicking::panic_bounds_check
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:159:5
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


**SOLUTION**

**Il manque une cellule au tableau, `array[3]` n'existe pas. Tentez `let array = [1, 2, 3, 4];`.**

### The good and the bad tuple

In [30]:
fn load_values(t: (u32, u32)) -> () { // RUN IT
    // ... Do some things
    dbg!(t);
}
load_values(my_new_tuple);

Error: cannot find value `my_new_tuple` in this scope

In [31]:
let my_new_tuple: (u32, u32, u32) = (1, 4, 11); // FIX IT AND RUN IT

**SOLUTION**

**La fonction load_values prends en argument un tuple de deux u32, c'est donc une erreur de lui envoyer un tuple de trois u32, cela ne ferait pas sens.**